<a href="https://colab.research.google.com/github/SLane35/CollabCompet/blob/master/EntityMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip uninstall deepmatcher

Uninstalling deepmatcher-0.1.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/deepmatcher-0.1.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/deepmatcher/*
Proceed (y/n)? y
  Successfully uninstalled deepmatcher-0.1.1


In [3]:
pip install git+https://github.com/anhaidgroup/deepmatcher.git

  Cloning https://github.com/anhaidgroup/deepmatcher.git to /tmp/pip-req-build-whna56km
  Running command git clone -q https://github.com/anhaidgroup/deepmatcher.git /tmp/pip-req-build-whna56km
  Created wheel for deepmatcher: filename=deepmatcher-0.1.1-py2.py3-none-any.whl size=55003 sha256=96b57336fc8867a7542052356538efd58372a65e85faa2de4013b5bebce20a25
  Stored in directory: /tmp/pip-ephem-wheel-cache-q7wfvfes/wheels/21/b3/11/dc933837985fcb4683f016ed48f88ae73f90f9b20c122fbedd
Successfully built deepmatcher


In [4]:

try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher


In [5]:
import deepmatcher as dm

In [7]:
import torch
torch.cuda.is_available()

True

In [6]:
!pip install -qqq recordlinkage

In [8]:
import pandas as pd
import recordlinkage
from recordlinkage.datasets import load_febrl1
df = load_febrl1(True)

In [9]:
dfNoLabels = load_febrl1()


In [10]:
matching_dictionary = df[1]

In [11]:
left_df = dfNoLabels.loc[matching_dictionary.get_level_values(0)]

In [12]:
right_df = dfNoLabels.loc[matching_dictionary.get_level_values(1)]

In [11]:
left_df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-223-dup-0,jamilla,wallner,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-dup-0,lachlan,berry,69,giblin street,killarney,bittern,4184,qld,19990219,7364009
rec-373-dup-0,deakin,sondergeld,231,goldfinch circuit,kooltuo,canterbury,2509,vic,19600210,2635962
rec-10-org,kayla,harrington,38,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-dup-0,mia,purdon,23,ramsay place,mirani,gar butt,2206,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-132-org,joshua,ingels,14,arabana street,tatyara,port lincoln,2325,vic,19080503,3603864
rec-498-dup-0,claire,crook,9,oyster farm,basedow street,NaN,3146,vic,NaN,2161417
rec-8-org,james,jesser,53,howe crescent,nuffield village,st clair,4215,vic,19120111,5961736
rec-222-dup-0,emiiyl,coleman,108,chewings street,berkeleyv lge,wellington point,2550,nsw,19421221,7206933


In [12]:
right_df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-132-dup-0,joshua,ingelbs,14,arabana street,tatyara,port lincoln,2352,vic,19080503,3603864
rec-498-org,claire,crook,9,basedow street,oyster farm,wanniassa,3146,vic,NaN,2161417
rec-8-dup-0,james,jeslser,53,howe crescent,nuffield v illage,st clair,4215,vic,19120111,5961736
rec-222-org,emiily,coleman,108,chewings street,berkeley vlge,wellington point,2550,nsw,19421221,7206933


In [13]:
left_df = left_df.drop(columns=['street_number', 'address_2', 'postcode', 'soc_sec_id'])
left_df

,given_name,surname,address_1,suburb,state,date_of_birth
rec-223-dup-0,jamilla,wallner,tullaroop street,st james,wa,19081209
rec-122-dup-0,lachlan,berry,giblin street,bittern,qld,19990219
rec-373-dup-0,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210
rec-10-org,kayla,harrington,maltby circuit,coolaroo,nsw,19150612
rec-227-dup-0,mia,purdon,ramsay place,gar butt,vic,19831024
...,...,...,...,...,...,...
rec-132-org,joshua,ingels,arabana street,port lincoln,vic,19080503
rec-498-dup-0,claire,crook,oyster farm,NaN,vic,NaN
rec-8-org,james,jesser,howe crescent,st clair,vic,19120111
rec-222-dup-0,emiiyl,coleman,chewings street,wellington point,nsw,19421221


In [14]:
right_df = right_df.drop(columns=['street_number', 'address_2', 'postcode', 'soc_sec_id'])
right_df

,given_name,surname,address_1,suburb,state,date_of_birth
rec-223-org,NaN,waller,tullaroop street,st james,wa,19081209
rec-122-org,lachlan,berry,giblin street,bittern,qld,19990219
rec-373-org,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210
rec-10-dup-0,kayla,harrington,maltby circuit,coolaroo,nsw,19150612
rec-227-org,luke,purdon,ramsay place,garbutt,vic,19831024
...,...,...,...,...,...,...
rec-132-dup-0,joshua,ingelbs,arabana street,port lincoln,vic,19080503
rec-498-org,claire,crook,basedow street,wanniassa,vic,NaN
rec-8-dup-0,james,jeslser,howe crescent,st clair,vic,19120111
rec-222-org,emiily,coleman,chewings street,wellington point,nsw,19421221


In [15]:
left_df_renamed = left_df.rename(columns={'given_name':'left_given_name', 'surname': 'left_surname', 'address_1':'left_address_1', 'suburb':'left_suburb', 'state':'left_state', 'date_of_birth':'left_date_of_birth'})

In [16]:
right_df_renamed = right_df.rename(columns={'given_name':'right_given_name', 'surname': 'right_surname',  'address_1':'right_address_1', 'suburb':'right_suburb', 'state':'right_state', 'date_of_birth':'right_date_of_birth'})

In [17]:
right_df_renamed['id'] = range(1, len(right_df_renamed)+1)

In [18]:
left_df_renamed['id'] = range(1, len(left_df_renamed)+1)

In [19]:
combined_df = pd.merge(left_df_renamed, right_df_renamed, on="id")

In [20]:
combined_df['label'] = 1
combined_df

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,NaN,waller,tullaroop street,st james,wa,19081209,1
1,lachlan,berry,giblin street,bittern,qld,19990219,2,lachlan,berry,giblin street,bittern,qld,19990219,1
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,1
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,1
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,luke,purdon,ramsay place,garbutt,vic,19831024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,joshua,ingelbs,arabana street,port lincoln,vic,19080503,1
496,claire,crook,oyster farm,NaN,vic,NaN,497,claire,crook,basedow street,wanniassa,vic,NaN,1
497,james,jesser,howe crescent,st clair,vic,19120111,498,james,jeslser,howe crescent,st clair,vic,19120111,1
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,emiily,coleman,chewings street,wellington point,nsw,19421221,1


In [21]:
right_df_shuffled = right_df.sample(frac=1).reset_index(drop=True)
right_df_shuffled

,given_name,surname,address_1,suburb,state,date_of_birth
0,jackson,cmielewski,angliss place,tweed heads south,vic,19141022
1,bordin,brody,canberra centre (next to medicare),figtree,qld,19510904
2,jamie,meixner,hibiscus crescent,atherton,nsw,19670124
3,bne,browen,tillyard drive,mentone,vic,19241201
4,mikayla,caruana,torrens street,dianella,qld,19590901
...,...,...,...,...,...,...
495,joshua,price-austin,kingsmillcstreet,mullumbimby,act,19210303
496,sam,white,pryor place,cmoo,nsw,19390109
497,eliza,keany,nicklin crescent,longreach,vic,19280802
498,emma,gildea,mccawley street,malvern,qld,19440429


In [22]:
right_df_shuffled = right_df_shuffled.rename(columns={'given_name':'right_given_name', 'surname': 'right_surname',  'address_1':'right_address_1', 'suburb':'right_suburb', 'state':'right_state', 'date_of_birth':'right_date_of_birth'})

In [23]:
right_df_shuffled['id'] = range(1, len(right_df_shuffled)+1)

In [24]:
combined_df_mixed = pd.merge(left_df_renamed, right_df_shuffled, on="id")
combined_df_mixed

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,jackson,cmielewski,angliss place,tweed heads south,vic,19141022
1,lachlan,berry,giblin street,bittern,qld,19990219,2,bordin,brody,canberra centre (next to medicare),figtree,qld,19510904
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,jamie,meixner,hibiscus crescent,atherton,nsw,19670124
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,bne,browen,tillyard drive,mentone,vic,19241201
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,mikayla,caruana,torrens street,dianella,qld,19590901
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,joshua,price-austin,kingsmillcstreet,mullumbimby,act,19210303
496,claire,crook,oyster farm,NaN,vic,NaN,497,sam,white,pryor place,cmoo,nsw,19390109
497,james,jesser,howe crescent,st clair,vic,19120111,498,eliza,keany,nicklin crescent,longreach,vic,19280802
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,emma,gildea,mccawley street,malvern,qld,19440429


In [25]:
combined_df_mixed['label'] = 0
combined_df_mixed

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,jackson,cmielewski,angliss place,tweed heads south,vic,19141022,0
1,lachlan,berry,giblin street,bittern,qld,19990219,2,bordin,brody,canberra centre (next to medicare),figtree,qld,19510904,0
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,jamie,meixner,hibiscus crescent,atherton,nsw,19670124,0
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,bne,browen,tillyard drive,mentone,vic,19241201,0
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,mikayla,caruana,torrens street,dianella,qld,19590901,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,joshua,price-austin,kingsmillcstreet,mullumbimby,act,19210303,0
496,claire,crook,oyster farm,NaN,vic,NaN,497,sam,white,pryor place,cmoo,nsw,19390109,0
497,james,jesser,howe crescent,st clair,vic,19120111,498,eliza,keany,nicklin crescent,longreach,vic,19280802,0
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,emma,gildea,mccawley street,malvern,qld,19440429,0


In [26]:
final_df = pd.concat([combined_df, combined_df_mixed], axis=0)
final_df

,left_given_name,left_surname,left_address_1,left_suburb,left_state,left_date_of_birth,id,right_given_name,right_surname,right_address_1,right_suburb,right_state,right_date_of_birth,label
0,jamilla,wallner,tullaroop street,st james,wa,19081209,1,NaN,waller,tullaroop street,st james,wa,19081209,1
1,lachlan,berry,giblin street,bittern,qld,19990219,2,lachlan,berry,giblin street,bittern,qld,19990219,1
2,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,3,deakin,sondergeld,goldfinch circuit,canterbury,vic,19600210,1
3,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,4,kayla,harrington,maltby circuit,coolaroo,nsw,19150612,1
4,mia,purdon,ramsay place,gar butt,vic,19831024,5,luke,purdon,ramsay place,garbutt,vic,19831024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,joshua,ingels,arabana street,port lincoln,vic,19080503,496,joshua,price-austin,kingsmillcstreet,mullumbimby,act,19210303,0
496,claire,crook,oyster farm,NaN,vic,NaN,497,sam,white,pryor place,cmoo,nsw,19390109,0
497,james,jesser,howe crescent,st clair,vic,19120111,498,eliza,keany,nicklin crescent,longreach,vic,19280802,0
498,emiiyl,coleman,chewings street,wellington point,nsw,19421221,499,emma,gildea,mccawley street,malvern,qld,19440429,0


In [27]:
final_df['id'] = range(1, len(final_df)+1)

In [28]:
import numpy as np
train,validate,test = np.split(final_df, [int(.8 * len(final_df)), int(.9 * len(final_df))])

In [29]:
from google.colab import drive
drive.mount('/drive')



Mounted at /drive


In [30]:
train.to_csv('/drive/My Drive/EntityMatching/train.csv')

In [31]:
validate.to_csv('/drive/My Drive/EntityMatching/validate.csv')
test.to_csv('/drive/My Drive/EntityMatching/test.csv')

In [4]:
processed_train, processed_validate, processed_test = dm.data.process(path='/drive/My Drive/EntityMatching', train='train.csv', validation='validate.csv', test='test.csv',ignore_columns=(''), left_prefix='left_',right_prefix='right_',label_attr='label',id_attr='id')

NameError: ignored

In [3]:
train_table = processed_train.get_raw_table()
train_table.head()

NameError: ignored

# New Section

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    processed_train,
    processed_validate,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)